In [1]:
import os
import keyring
# import warnings
# warnings.filterwarnings('ignore')
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_windows')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [2]:
# web search and code execution tool
from typing import Annotated
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)
python_repl_tool = PythonREPLTool()

In [3]:
# recent stock prices and financial analyzing tool with yfinance
import yfinance as yf 
from datetime import datetime, timedelta
import pandas as pd 

ticker = yf.Ticker('NVDA')
historical_prices = ticker.history(period='5d', interval='1d')

last_5_days_close = historical_prices['Close'].tail(5)
last_5_days_close_dict = {date.strftime('%Y-%m-%d'): price for date, price in last_5_days_close.items()}

# annual and quarterly financial statements data
last_5_days_close_dict

{'2024-11-22': 141.9499969482422,
 '2024-11-25': 136.02000427246094,
 '2024-11-26': 136.9199981689453,
 '2024-11-27': 135.33999633789062,
 '2024-11-29': 138.25}

In [4]:
from langchain.tools import BaseTool, StructuredTool, tool
import yfinance as yf 
from datetime import datetime, timedelta
import pandas as pd 

# financial analysis tool
@tool
def stock_analysis(ticker: str) -> str:
    """ 
    Total financial analysis on the figen stock ticker.
    Recent stock prices, financial statements, growth rates, valuation and ratios.
    Args
        ticker: str - stock ticker
        return: str -
    """
    def format_number(number):
        if number is None or pd.isna(number):
            return 'N/A'
        return f"{number:,.0f}"
    
    def format_financial_summary(financials):
        summary = {}
        for date, data in financials.items():
            date_str = date.strftime('%Y-%m-%d')
            summary[date_str] = {
                'Total revenue': format_number(data.get('TotalRevenue')),
                'Operating Income': format_number(data.get('OperatingIncome')),
                'Net Income': format_number(data.get('NetIncome')),
                'EBITDA': format_number(data.get('EBITDA')),
                'Diluted EPS': f"${data.get('DilutedEPS'):.2f}" if pd.notna(data.get('DilutedEPS')) else "N/A"
            }
        return summary
    
    ticker = yf.Ticker(ticker)
    historical_prices = ticker.history(period='5d', intervals='1d')
    
    last_5_days_close = historical_prices['Close'].tail(5)
    last_5_days_close_dict = {date.strftime('%Y-%m-%d'): price for date, price in last_5_days_close.items()}
    
    # annual and quarterly financial statements
    annual_financials = ticker.get_financials()
    quarterly_financials = ticker.get_financials(freq='quarterly')
    
    return str({
        "Recent 5 days close prices": last_5_days_close_dict,
        "annual financial statements summary": format_financial_summary(annual_financials),
        "quarterly financial statements summary": format_financial_summary(quarterly_financials),
    })

In [6]:
# Supervisor agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

members = ['Researcher', 'Stock_Analyzer', 'Chart_Generator']
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

# our team supervisor is an LLM node. It just picks the next agent to process
# and decided when the work is completed
options = ['FINISH'] + members

class routeResponse(BaseModel):
    next: Literal['FINISH', 'Researcher', 'Stock_Analyzer', 'Chart_Generator']
    
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        )
    ]
).partial(options=", ".join(options), members=", ".join(members))
     
llm = ChatOpenAI(model='gpt-4o-mini')

def supervisor_agent(state):
    supervisor_chain = (
        prompt | llm.with_structured_output(routeResponse)
    )
    return supervisor_chain.invoke(state)

In [7]:
# define sub agent function
from langchain_core.messages import HumanMessage

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["messages"][-1].content, name=name)]}